In [1]:
#########################################################
## Stat 202A - Homework 4
## Author: Chaojie Feng
## Date : Oct.25,2018
## Description: This script implements logistic regression
## using iterated reweighted least squares using the code 
## we have written for linear regression based on QR 
## decomposition
#########################################################

#############################################################
## INSTRUCTIONS: Please fill in the missing lines of code
## only where specified. Do not change function names, 
## function inputs or outputs. You can add examples at the
## end of the script (in the "Optional examples" section) to 
## double-check your work, but MAKE SURE TO COMMENT OUT ALL 
## OF YOUR EXAMPLES BEFORE SUBMITTING.
##
## Very important: Do not use the function "setwd" anywhere
## in your code. If you do, I will be unable to grade your 
## work since R will attempt to change my working directory
## to one that does not exist.
#############################################################

##################################
## Function 1: QR decomposition ##
##################################

myQR <- function(A){
  
  ## Perform QR decomposition on the matrix A
  ## Input: 
  ## A, an n x m matrix
  
  ########################
  ## FILL IN CODE BELOW ##
  ########################  
    
    n = dim(A)[1]
    m = dim(A)[2]
    if(n<m){
        return(-1)
    }
    R = A
    Q = diag(rep(1,n))
    for(k in 1:(m-1)){
        x = array(0,dim = c(n,1))
        x[k:n,1] = R[k:n,k]
        v = x
        v[k,1] = x[k,1] + sign(x[k,1])*sqrt(sum(x^2))
        s = sqrt(sum(v^2))
        if(s != 0){
            u = v/s
            R <- R - (2 * u %*% (t(u) %*% R))
            Q <- Q - (2 * u %*% (t(u) %*% Q))
            }
        
        }

  ## Function should output a list with Q.transpose and R
  ## Q is an orthogonal n x n matrix
  ## R is an upper triangular n x m matrix
  ## Q and R satisfy the equation: A = Q %*% R
  return(list("Q" = t(Q), "R" = R))
  
}

###############################################
## Function 2: Linear regression based on QR ##
###############################################

myLM <- function(X, Y){
  
  ## Perform the linear regression of Y on X
  ## Input: 
  ## X is an n x p matrix of explanatory variables
  ## Y is an n dimensional vector of responses
  ## Use myQR inside of this function
  
  ########################
  ## FILL IN CODE BELOW ##
  ########################  
    n = dim(X)[1]
    p = dim(X)[2]
    
    Z = cbind(X,Y)
    
    R = myQR(Z)$R
    R1 = R[1:p,1:p]
    Y1 = R[1:p,(p+1)]
    
    beta_ls = solve(R1,Y1)
  
  ## Function returns the 1 x (p) vector beta_ls, 
  ## the least squares solution vector
  return(beta_ls)
  
  
}

######################################
## Function 3: Logistic regression  ##
######################################

## Expit/sigmoid function
expit <- function(x){
  1 / (1 + exp(-x))
}

myLogisticSolution <- function(X, Y){

  ########################
  ## FILL IN CODE BELOW ##
  ########################
    
    r = dim(X)[1]
    c = dim(X)[2]
    
    beta = rep(0,c)
    epsilon = 1e-6
    
    while(TRUE){
        eta = X %*% beta
        pr = expit(eta)
        w = pr*(1-pr)
        z = eta + (Y-pr)/w
        sw = sqrt(w)
        mw = matrix(sw,nrow = r, ncol = c)
        x_work = mw * X
        y_work = sw * z
        beta_update = myLM(x_work,y_work)
        if (sqrt(sum((beta - beta_update)^2)) < epsilon) {
            break
        }
        beta <- beta_update
        print(beta)
    }
    
    return(beta)
    
}

##################################################
## Function 4: Eigen decomposition based on QR  ##
##################################################
myEigen_QR <- function(A, numIter = 1000) {
  
  ## Perform PCA on matrix A using your QR function, myQRC.
  ## Input:
  ## A: Square matrix
  ## numIter: Number of iterations
  
  ########################
  ## FILL IN CODE BELOW ##
  ######################## 
    A_copy = A
    r = dim(A)[1]
    c = dim(A)[2]
    v = matrix(rnorm(r^2),nrow = r)
    for(i in 1:numIter){
        Q = unlist(myQR(v)$Q)
        v = A_copy %*% Q
    }
    Q = myQR(v)$Q
    R = myQR(v)$R
    
    lambda <- diag(R)
    Q <- Q[, c(order(-lambda))]
    lambda <- lambda[order(-lambda)]
  
  ## Function should output a list with D and V
  ## D is a vector of eigenvalues of A
  ## V is the matrix of eigenvectors of A (in the 
  ## same order as the eigenvalues in D.)
  
  return(list("D" = lambda, "V" = Q))
}

###################################################
## Function 5: PCA based on Eigen decomposition  ##
###################################################
myPCA <- function(X) {
  
  ## Perform PCA on matrix A using your eigen decomposition.
  ## Input:
  ## X: Input Matrix with dimension n * p
    
  A <- t(X) %*% X
  Q <- unlist(myEigen_QR(A)$V)
  Z <- X %*% Q


  ## Output : 
  ## Q : basis matrix, p * p which is the basis system.
  ## Z : data matrix with dimension n * p based on the basis Q.
  ## It should match X = Z %*% Q.T. Please follow lecture notes.

  return(list("Q" = Q, "Z" = Z))
}


